<center style = "font-family: Trebuchet MS, sans-serif;">
    <h1 style = "background-color:#E39DE3; color:white;"> RECOMMENDER SYSTEMS - ASSIGNMENT 01 </h1>
    <hr style = "background-color:#E39DE3;">
</center>
<h3 style = "color:#E39DE3; font-family:Trebuchet MS, sans-serif;"> 
    <b>NAME: </b>Sondos Suhail <br>
    <b>ID: </b>20K-1057
</h3>
<hr style = "background-color:#E39DE3;">
<h3 style = "color:#E39DE3; font-family:Trebuchet MS, sans-serif;">  Imported Libraries </h3>

In [1]:
import pandas as pd
import numpy as np
import math
import nltk

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h3> Task 01 </h3>

<p >Write a Python (or Java) program to implement a user-based Collaborative Filtering Recommender System that takes a movie name as input and recommends top-5 movies to the current user. Use Cosine Similarity for finding neighborhoods and mean-centered prediction function. </p>

<b>UTILITY FUNCTIONS

In [2]:
def remove_year(mov_title):
    x = []
    for i in mov_title:
        y = i[:-6]
        y = y.strip()
        x.append(y)
        
    return x

In [3]:
def predict_rating(user_id, movie_title, ratings_matrix, user_sim, user_means):
    if not pd.isnull(ratings_matrix.loc[user_id, movie_title]):
        return "User already rated this movie"
    
    user_ratings = ratings_matrix.loc[user_id].dropna()
    user_mean = user_means[user_id]
    
    sim_scores = user_sim[user_id]
    sim_ratings = ratings_matrix.drop(user_id).loc[:, movie_title]
    if sim_ratings.isna().all():
        return "No ratings for this movie from similar users"
    
    sim_ratings_centered = sim_ratings - user_means
    weighted_sum = (sim_ratings_centered * sim_scores).sum()
    weighted_sum /= sim_scores.sum()    
    predicted_rating = weighted_sum + user_mean
    
    return predicted_rating


<b>LOADING ALL DATASETS INTO DATAFRAMES

In [4]:
ratings = pd.read_csv('ratings.csv')
ratings = ratings.drop('timestamp', axis = 1)

movies = pd.read_csv('movies.csv')
movies = movies.drop('genres', axis = 1)
movies['title'] = remove_year(movies['title'])

df = pd.merge(ratings, movies, on='movieId')
df

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,5,1,4.0,Toy Story
2,7,1,4.5,Toy Story
3,15,1,2.5,Toy Story
4,17,1,4.5,Toy Story
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon
100832,610,160527,4.5,Sympathy for the Underdog
100833,610,160836,3.0,Hazard
100834,610,163937,3.5,Blair Witch


In [5]:
ratings_matrix = df.pivot_table(index='userId', columns='title', values='rating')
ratings_matrix

title,,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [6]:
user_means = ratings_matrix.mean(axis=1)
user_means

userId
1      4.373362
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.656561
607    3.786096
608    3.129394
609    3.270270
610    3.687211
Length: 610, dtype: float64

In [7]:
ratings_norm = ratings_matrix.sub(user_means, axis = 0)
ratings_norm

title,,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.373362,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.370606,0.370606,NaN,NaN,NaN


In [8]:
user_sim = cosine_similarity(ratings_norm.fillna(0))
user_sim

array([[ 1.00000000e+00,  1.27914569e-03,  5.90326277e-04, ...,
         7.70011936e-02, -2.66407544e-02,  4.36692402e-03],
       [ 1.27914569e-03,  1.00000000e+00,  0.00000000e+00, ...,
        -5.94868997e-03, -6.00909967e-02,  2.50431914e-02],
       [ 5.90326277e-04,  0.00000000e+00,  1.00000000e+00, ...,
        -1.31465423e-02,  0.00000000e+00,  1.96085733e-02],
       ...,
       [ 7.70011936e-02, -5.94868997e-03, -1.31465423e-02, ...,
         1.00000000e+00,  5.09349787e-02,  5.91143422e-02],
       [-2.66407544e-02, -6.00909967e-02,  0.00000000e+00, ...,
         5.09349787e-02,  1.00000000e+00, -1.24804375e-02],
       [ 4.36692402e-03,  2.50431914e-02,  1.96085733e-02, ...,
         5.91143422e-02, -1.24804375e-02,  1.00000000e+00]])

In [9]:
movie_title = 'Pocahontas'
user_id = 2
predicted_rating = predict_rating(user_id, movie_title, ratings_matrix, user_sim, user_means)
print(f"Predicted rating of {movie_title} for user {user_id}: {round(predicted_rating, 2)}")

Predicted rating of Pocahontas for user 2: 3.97
